In [28]:
import pandas as pd

# Загрузка данных из файлов CSV
assessments = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-v-ustimova/mini_proj_1/assessments.csv')
courses = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-v-ustimova/mini_proj_1/courses.csv')
student_assessment = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-v-ustimova/mini_proj_1/studentAssessment.csv')
student_registration = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-v-ustimova/mini_proj_1/studentRegistration.csv')

# Объединение таблиц assessments, courses, student_assessment и student_registration по столбцам code_module и code_presentation
merged_data = pd.merge(assessments, courses, on=['code_module', 'code_presentation'])
merged_data = pd.merge(merged_data, student_assessment, on='id_assessment')
merged_data = pd.merge(merged_data, student_registration, on=['code_module', 'code_presentation', 'id_student'])

In [6]:
# Первые строки соединенной таблицы
merged_data.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight,module_presentation_length,id_student,date_submitted,is_banked,score,date_registration,date_unregistration
0,AAA,2013J,1752,TMA,19.0,10.0,268,11391,18,0,78.0,-159.0,NaN
1,AAA,2013J,1753,TMA,54.0,20.0,268,11391,53,0,85.0,-159.0,NaN
2,AAA,2013J,1754,TMA,117.0,20.0,268,11391,115,0,80.0,-159.0,NaN
3,AAA,2013J,1755,TMA,166.0,20.0,268,11391,164,0,85.0,-159.0,NaN
4,AAA,2013J,1756,TMA,215.0,30.0,268,11391,212,0,82.0,-159.0,NaN


In [8]:
#ЗАДАНИЕ №1 

# Студенты, у которых оценка >= 40
filtered_data = merged_data[merged_data['score'] >= 40]

# Группировка данных по студентам, количество успешно сданных курсов
grouped_data = filtered_data.groupby('id_student')['code_module'].nunique().reset_index()

# Студенты с одним успешно сданным курсом
filtered_grouped_data = grouped_data[grouped_data['code_module'] == 1]

# Количество студентов
count = len(filtered_grouped_data)

# Результат
print("Количество студентов, успешно сдавших только один курс:", count)

#ОТВЕТ:21428

Количество студентов, успешно сдавших только один курс: 21428


In [12]:
#ЗАДАНИЕ №2 

# Фильтрация по экзаменам
filtered_data = merged_data[merged_data['assessment_type'] == 'Exam']

# Группировка данных по коду предмета, идентификатору экзамена, подсчет средней завершаемости
grouped_data = filtered_data.groupby(['code_module', 'id_assessment'])['score'].mean().reset_index()

# Экзамен с самой низкой завершаемостью
low_exam = grouped_data[grouped_data['score'] == grouped_data['score'].min()]

# Экзамен с самой высокой завершаемостью
high_exam = grouped_data[grouped_data['score'] == grouped_data['score'].max()]

# Результат
print("Самый сложный экзамен:")
print(low_exam)

print("Самый простой экзамен:")
print(high_exam)

# ОТВЕТ:

# Самый сложный экзамен:
#  code_module  id_assessment      score
#          DDD          25340  56.353821

# Самый простой экзамен:
#  code_module  id_assessment      score
#          CCC          24290  69.970549

Самый сложный экзамен:
  code_module  id_assessment      score
2         DDD          25340  56.353821
Самый простой экзамен:
  code_module  id_assessment      score
0         CCC          24290  69.970549


In [13]:
#ЗАДАНИЕ №3

# Фильтрация по экзаменам
filtered_data_2 = merged_data[merged_data['assessment_type'] == 'Exam']

# Студенты, у которых оценка >= 40
filtered_data_2 = filtered_data_2[filtered_data_2['score'] >= 40]

# Макс дата сдачи
max_date_submitted = filtered_data_2.groupby(['code_module', 'id_student'])['date_submitted'].max().reset_index()

# Средний срок сдачи 
average_completion_time = max_date_submitted.groupby('code_module')['date_submitted'].mean().reset_index()

# Результат
print("Средний срок сдачи экзаменов по каждому предмету:")
print(average_completion_time)

# ОТВЕТ:

# Средний срок сдачи экзаменов по каждому предмету:
#  code_module  date_submitted
#         CCC      239.345217
#         DDD      237.978566

Средний срок сдачи экзаменов по каждому предмету:
  code_module  date_submitted
0         CCC      239.345217
1         DDD      237.978566


In [24]:
#ЗАДАНИЕ №4

# Группировка по коду предмета, количество регистраций
popular_courses = merged_data.groupby('code_module')['id_student'].nunique().reset_index()

# Сортировка по количеству регистраций в порядке убывания, ТОП-3 
top_popular_courses = popular_courses.sort_values(by='id_student', ascending=False).head(3)

# Студенты, отменившие регистрацию
churned_students = merged_data[merged_data['date_unregistration'].notnull()]['id_student'].unique()

# Фильтрация по отмененным регистрациям
churned_data = merged_data[merged_data['id_student'].isin(churned_students)]

# Группировка по коду предмета, количество отмен регистрации
churn_courses = churned_data.groupby('code_module')['id_student'].nunique().reset_index()

# Сортировка по количеству отмен регистрации в порядке убывания, ТОП-3
top_churn_courses = churn_courses.sort_values(by='id_student', ascending=False).head(3)

# Результат
print("Самые популярные предметы (ТОП-3) по количеству регистраций:")
print(top_popular_courses)
print("Предметы с самым большим оттоком (ТОП-3):")
print(top_churn_courses)

# ОТВЕТ:
#Самые популярные предметы (ТОП-3) по количеству регистраций:
#  code_module  id_student
#         FFF        6058
#         BBB        5960
#         DDD        4699

#Предметы с самым большим оттоком (ТОП-3):
#  code_module  id_student
#         DDD        1194
#         FFF        1188
#         CCC        1041

Самые популярные предметы (ТОП-3) по количеству регистраций:
  code_module  id_student
5         FFF        6058
1         BBB        5960
3         DDD        4699
Предметы с самым большим оттоком (ТОП-3):
  code_module  id_student
3         DDD        1194
5         FFF        1188
2         CCC        1041


In [36]:
# Вывод уникальных значений из столбца code_presentation. Далее будет считаться,что семестр "J" соответствует 
#январю-июню, а семестр "B" соответствует июлю-декабрю
unique_values = merged_data['code_presentation'].unique()

print(unique_values)

['2013J' '2014J' '2013B' '2014B']


In [38]:
#ЗАДАНИЕ №5

# Извлечение года и семестра из столбца "code_presentation"
merged_data['year'] = merged_data['code_presentation'].str[:4]
merged_data['semester'] = merged_data['code_presentation'].str[4:]

# Преобразование семестра в месяц
merged_data.loc[merged_data['semester'] == 'J', 'month'] = 1
merged_data.loc[merged_data['semester'] == 'B', 'month'] = 7

# "date" на основе года и месяца
merged_data['date'] = pd.to_datetime(merged_data['year'] + '-' + merged_data['month'].astype(int).astype(str) + '-01')

# Фильтрация с начала 2013 - конец 2014 года и типу теста "Exam"
filtered_data_3 = merged_data[(merged_data['date'] >= '2013-01-01') & (merged_data['date'] <= '2014-12-31') & (merged_data['assessment_type'] == 'Exam')]

# Группировка по семестру и подсчет завершаемости курсов
completion_rate = filtered_data_3.groupby('code_presentation')['score'].apply(lambda x: (x >= 40).sum() / len(x))

# Семестр с самой низкой завершаемостью
lowest_completion_rate_semester = completion_rate.idxmin()

# Группировка по семестру и вычисление среднего времени сдачи курсов
average_completion_time = filtered_data_3.groupby('code_presentation')['date'].apply(lambda x: (x.max() - x.min()).days)

# Семестр с самым долгим средним временем сдачи курсов
longest_completion_time_semester = average_completion_time.idxmax()

# Результат
print("Семестр с самой низкой завершаемостью курсов:", lowest_completion_rate_semester)
print("Семестр с самым долгим средним временем сдачи курсов:", longest_completion_time_semester)

# ОТВЕТ:
#Семестр с самой низкой завершаемостью курсов: 2013B
#Семестр с самым долгим средним временем сдачи курсов: 2013B

Семестр с самой низкой завершаемостью курсов: 2013B
Семестр с самым долгим средним временем сдачи курсов: 2013B
